<ceter > 
    <h1> Efficient NEt B5 Kfold over Optuna</h1>
    </center>

In [ ]:
!pip install --quiet ../input/kerasapplications
!pip install --quiet ../input/efficientnet-source-code

In [ ]:
# import libraries
import numpy as np 
import pandas as pd 
from os.path import join
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import os

import IPython.display as display

from PIL import Image
import warnings
#warnings.filterwarnings("ignore")

import random as rd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Input,optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization, Activation, Input, GlobalAveragePooling2D, concatenate,Lambda
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping,TensorBoard
from tensorflow.keras.applications import EfficientNetB0,EfficientNetB4,EfficientNetB3, Xception
import efficientnet.tfkeras as efn

from tensorflow.keras.mixed_precision import experimental as mixed_precision
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,roc_curve,multilabel_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import itertools

import optuna

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
def plot_confusion_matrix(y_true, y_pred, class_names,title="Confusion matrix",normalize=False,onehot = False):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.
    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    if onehot :
        cm = confusion_matrix([y_i.argmax() for y_i in y_true], [y_ip.argmax() for y_ip in y_pred])
    else:
        cm = confusion_matrix(y_true, y_pred)
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2) if normalize else cm

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
SampleSubmission = "../input/cassava-leaf-disease-classification/sample_submission.csv"
labels_desease  = "../input/cassava-leaf-disease-classification/label_num_to_disease_map.json"
traintablepath = "../input/cassava-leaf-disease-classification/train.csv"

train_tfr_path = "../input/cassava-leaf-disease-classification/train_tfrecords"
test_tfr_path = "../input/cassava-leaf-disease-classification/train_tfrecords"

train_img_path = "../input/cassava-leaf-disease-classification/train_images"
test_img_path  = "../input/cassava-leaf-disease-classification/test_images"

models_path  = ""

## Image Augmentation

In [ ]:
img_augmentation = tf.keras.Sequential(
    [
        #tf.keras.layers.experimental.preprocessing.RandomCrop(image_size, image_size), #during inference will work like a resize layer
        #tf.keras.layers.experimental.preprocessing.RandomTranslation((-.1,.1), (-.1,-.1)),
        tf.keras.layers.experimental.preprocessing.RandomZoom((-.1,-.1)),
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
        tf.keras.layers.experimental.preprocessing.RandomContrast(0.1),
    ])

In [ ]:
class ImageCustomGenerator(tf.keras.utils.Sequence):
    """
     df_table, dataframe
     col_name, column name for file images
     col_label, column label
     indexs, index to extract from dataframe
     path_files, folder path where images are stored
     batch_size,
     target_size, output images
    """
    def __init__(self, 
                 df_table,
                 col_name,
                 col_label,
                 path_files,
                 batch_size,
                 target_size,
                 augmented_seq=None,
                 indexs=None,
                 shuffle = False,
                 TEST = False
                ):
        self.df_table = df_table
        self.augmented_seq = augmented_seq
        self.indexs = indexs if not TEST else df_table.index.tolist()
        self.col_name = col_name
        self.col_label = col_label
        self.path_files = path_files
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.TEST = TEST
        self.n_class = df_table[col_label].nunique() if not TEST else 1
        self.on_epoch_end()
        self.__info()
        
    def __info(self):
        print("Custom Image generator {:4d} images".format(len(self.indexs)))

    def __len__(self):
        #batches per epochs
        return len(self.indexs) // self.batch_size + 1 if len(self.indexs)%self.batch_size != 0 else len(self.indexs) // self.batch_size

    def __getitem__(self, index):
        indexs_step = self.index_gen[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indexs[k] for k in indexs_step]
        X, y = self.__get_data(batch)
        return X, y

    def on_epoch_end(self):
        self.index_gen = np.arange(len(self.indexs))
        if self.shuffle == True and self.TEST == False:
            np.random.shuffle(self.index_gen)
            
    def __get_data(self, batch):
        X = []
        if not self.TEST:
            y = self.df_table.iloc[batch,:][self.col_label].to_numpy().astype(int)
            file_list = self.df_table.iloc[batch,:][self.col_name]
            file_list = file_list.map(lambda x: os.path.join(self.path_files,x))

            for name in file_list:
                img_ = tf.image.resize(np.array(np.array(Image.open(name))), self.target_size, method = "bilinear")
                X.append(img_)
            return tf.stack(X), tf.stack(y)
        
        else:#################### TEST BEHAVIOR ################
            file_list = self.df_table.iloc[batch,:][self.col_name]
            file_list = file_list.map(lambda x: os.path.join(self.path_files,x))
            for name in file_list:
                img_ = tf.image.resize(np.array(np.array(Image.open(name))),[600,800], method = "bilinear")
                img_ = tf.image.random_crop(img_, [*self.target_size,3])
                X.append(img_)
            return  self.augmented_seq(tf.stack(X))/255,  tf.zeros([len(batch)], tf.float32)
            
    

## EfficientNet Model B5

In [ ]:
def create_model(model_id,CLASSES=5,IMAGE_SIZE=(450,450)):
  if model_id==0:
    pretrained_model = tf.keras.applications.Xception(weights = None, input_shape=[*IMAGE_SIZE, 3], include_top=False)
  elif model_id==1:
    pretrained_model = efn.EfficientNetB5(weights = None, include_top = False,input_shape=[*IMAGE_SIZE, 3] )
  elif model_id ==2:
    pretrained_model = efn.EfficientNetB0(weights = None, include_top = False,input_shape=[*IMAGE_SIZE, 3] )
  
  pretrained_model.trainable = True
  model = tf.keras.Sequential([
    pretrained_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(CLASSES, activation='softmax')
  ])
  model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=[tf.keras.metrics.CategoricalAccuracy(),
                               tf.keras.metrics.PrecisionAtRecall(recall=0.8),
                               tf.keras.metrics.AUC(name='auc', multi_label= True),
                               tf.keras.metrics.Recall(name='recall')])
  return model

## Creating Generator

In [ ]:
train_table = pd.read_csv(traintablepath)
train_table_gen = train_table.copy()
train_table_gen["label"] = train_table_gen["label"].astype(str)
train_datagen = ImageDataGenerator(rescale=1./255,).flow_from_dataframe(
                dataframe = train_table_gen,
                directory = train_img_path,
                x_col="image_id",
                y_col="label",
                target_size=(450, 450),
                batch_size=32,
                shuffle=False,
                class_mode='categorical',
                validate_filenames=False,)

In [ ]:
train_table.head()

## Inference over the whole dataset

In [ ]:
import glob
path_model_efnB5 = glob.glob("../input/cassava-efficientnetb5-450px/*.hdf5") 
correct = train_table['label']
results = []

for model_k in path_model_efnB5:
    tf.keras.backend.clear_session() 
    model = create_model(1)
    model.load_weights(model_k)
    predict_k = model.predict(train_datagen, verbose = 1)
    df = pd.DataFrame(data= predict_k, columns = ["c0", "c1", "c2", "c3", "c4"])
    results.append(df)

In [ ]:
results

In [ ]:
def total_inference(dfs, w0, w1, w2, w3,w4):
    base= w0 + w1+ w2+ w3+w4
    total = dfs[0]*w0 + dfs[1]*w1 + dfs[2]*w2 + dfs[3]*w3 + dfs[4]*w4
    total /= base
    total = total.to_numpy()
    return np.argmax(total, axis=1)

In [ ]:
final_inference = total_inference(results, .2,.2,.2,.2,.2)
accuracy_score(correct, final_inference)

## Optuna Optimization

In [ ]:
# You can increase iteration number.
iteration = 4000

optuna.logging.disable_default_handler() # not display log
#optuna.logging.enable_default_handler() # display log

In [ ]:
%%time

def objective(trial):
    r_min = 0
    r_max = 1
    w0 = trial.suggest_uniform('w0', r_min, r_max)
    w1 = trial.suggest_uniform('w1', r_min, r_max)
    w2 = trial.suggest_uniform('w2', r_min, r_max)
    w3 = trial.suggest_uniform('w3', r_min, r_max)
    w4 = trial.suggest_uniform('w4', r_min, r_max)

    pred = total_inference(results, w0, w1, w2, w3,w4)
    score = accuracy_score(correct, pred)
    #print('a:%1.3f,b:%1.3f,c:%1.3f,d:%1.3f,e:%1.3f,score %1.3f' % (a,b,c,d,e,score))
    return score
SEED=5050
#study = optuna.create_study(direction='maximize')
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=SEED))
#study.optimize(objective, n_trials=iteration)

In [ ]:
study.best_value

In [ ]:
study.best_params

In [ ]:
plt.plot([trial.value for trial in study.trials])
plt.grid()
plt.show()

In [ ]:
plt.plot([trial.params['w0'] for trial in study.trials], label='w0')
plt.plot([trial.params['w1'] for trial in study.trials], label='w1')
plt.plot([trial.params['w2'] for trial in study.trials], label='w2')
plt.plot([trial.params['w3'] for trial in study.trials], label='w3')
plt.plot([trial.params['w4'] for trial in study.trials], label='w4')
plt.legend()
plt.grid()
plt.show()

In [ ]:
from optuna.visualization import plot_optimization_history
plot_optimization_history(study)

In [ ]:
%time
import glob
path_model_efnB5 = glob.glob("../input/cassava-efficientnetb5-450px/*.hdf5")
#path_model_xcept = glob.glob("../input/cassava-xception-450px/*.hdf5")
#path_model_efnB0 = glob.glob("../input/cassava-efficientnetb0-450/*.hdf5")
# 0.8912931719399916
weights = {0: 0.0004230151176771213,
             1: 0.5114438069306837,
             2: 0.06629172919594499,
             3: 0.42302383984983993,
             4: 0.742831400357499}

TTA_n = 4

#table_test_dummy = sampSum.copy()
table_test_dummy = pd.DataFrame()
table_test_dummy['image_id'] = list(os.listdir(test_img_path))
table_test_dummy['label'] = 0

n_splits = 5
#### Just for testing purpose
test_gen = ImageCustomGenerator(
                 df_table = table_test_dummy, # sampSum
                 col_name = "image_id",
                 col_label = "label",
                 augmented_seq = img_augmentation, 
                 path_files= test_img_path ,       # test_img_path
                 batch_size = 6,
                 target_size = (450, 450),
                 TEST = True)


y_predict = np.zeros((len(table_test_dummy),5))     #np.zeros((len(sampSum),5))
model_name = [  "Xception","EfficientNet B5","EfficientNet B0"]
for id_model, list_kfold_models in enumerate([path_model_efnB5 ],1):
    y_predict_model = np.zeros((len(table_test_dummy),5))
    print("Inference by model - {}".format(model_name[id_model]))
    for ki, model_kfold_path in enumerate(list_kfold_models):
        print("{} - {} Kfold pre trained".format(model_name[id_model], ki))
        tf.keras.backend.clear_session() 
        model = create_model(id_model)
        model.load_weights(model_kfold_path)
        y_predict_tta = np.zeros((len(table_test_dummy),5))
        for t in range(TTA_n):
            print("N°{} TTA".format(t))
            y_predict_tta += model.predict(test_gen, verbose=1)
            
        y_predict_model += y_predict_tta/TTA_n*weights[ki]
    y_predict += y_predict_model

    
y_predict = np.argmax(y_predict, axis = 1)
table_test_dummy["label"] = y_predict
table_test_dummy.to_csv("submission.csv", index=False)
table_test_dummy.head()